In [1]:
import pandas as pd
pd.options.display.float_format = "{:,.2f}".format

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What are we going to cover?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">How can we view the entire history of a Property's value using the LUSID APIs via the Python SDK?</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">How can we use the Property history to understand what the value of the Property is at any point in bi-temporal space?</p></li>
</ul>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">Retrieving the History of a Time-Variant Property's Values</p>

In [2]:
import pprint

import lusid
from lusidjam import RefreshingToken
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame

api_factory = lusid.utilities.ApiClientFactory(token=RefreshingToken())
instruments_api = api_factory.build(lusid.api.InstrumentsApi)


In [3]:
instruments_api.upsert_instruments(
    scope="FinbourneUniversity",
    request_body={
        "instr_1": lusid.InstrumentDefinition(
            name="MICROSOFT CORP",
            identifiers={
                "Figi": lusid.InstrumentIdValue(
                    value="BBG000BPH459"
                )
            }
        )
    }
)

{'failed': {},
 'href': None,
 'links': [{'description': None,
            'href': 'https://demo.lusid.com/api/api/schemas/entities/UpsertInstrumentsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://demo.lusid.com/app/insights/logs/0HMSTB5RBVOFJ:0000008D',
            'method': 'GET',
            'relation': 'RequestLogs'}],
 'metadata': {'actions': [{'description': 'The request identifiers of Unchanged Instruments',
 'identifier_type': 'RequestId',
 'identifiers': ['instr_1'],
 'type': 'UnchangedInstruments'}]},
 'values': {'instr_1': {'asset_class': 'Unknown',
                        'dom_ccy': 'ZZZ',
                        'href': 'https://demo.lusid.com/api/api/instruments/LusidInstrumentId/LUID_00003SOT?scope=FinbourneUniversity',
                        'identifiers': {'Figi': 'BBG000B

In [4]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.delete_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisation")
except:
    pass

In [5]:
def upsert_instrument_properties(value, effective_from=None, effective_until=None):
    
    if value is None:
        property_value = None
    else:
        property_value = lusid.PropertyValue(
            metric_value=lusid.MetricValue(
                value=value,
                unit="USD"
            )
        )

    print(instruments_api.upsert_instruments_properties(
        scope="FinbourneUniversity",
        upsert_instrument_property_request=[
            lusid.UpsertInstrumentPropertyRequest(
                identifier_type="Figi",
                identifier="BBG000BPH459",
                properties=[
                    lusid.ModelProperty(
                        key="Instrument/FinbourneUniversity/MarketCapitalisationv2",
                        value=property_value,
                        effective_from=effective_from,
                        effective_until=effective_until
                    )
                ]
            )
        ]
    ))




In [6]:
property_definitions_api = api_factory.build(lusid.api.PropertyDefinitionsApi)

try:
    property_definitions_api.get_property_definition(
        domain="Instrument",
        scope="FinbourneUniversity",
        code="MarketCapitalisationv2")
except:
    issuer_prop_definition = property_definitions_api.create_property_definition(
        create_property_definition_request=lusid.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope="FinbourneUniversity",
            code="MarketCapitalisationv2",
            display_name="Market Capitalisation",
            property_description="The market capitalisation of the instrument",
            data_type_id=lusid.ResourceId(
                scope="system",
                code="currencyAndAmount"
            ),
            life_time="TimeVariant",
            constraint_style="Property"
        )
    )
    
    upsert_instrument_properties(780360000000)
    upsert_instrument_properties(1200000000000, effective_from="2020-01-01")
    upsert_instrument_properties(1680000000000, effective_from="2021-01-01", effective_until="2022-01-01")
    upsert_instrument_properties(None, effective_from=None, effective_until="2018-12-31")
    upsert_instrument_properties(None, effective_from="2022-01-01")

In [7]:
result = instruments_api.get_instrument_property_time_series(
    identifier_type="Figi",
    identifier="BBG000BPH459",
    scope="FinbourneUniversity",
    property_key="Instrument/FinbourneUniversity/MarketCapitalisationv2")

result_df = lusid_response_to_data_frame(result)
result_df = result_df.sort_values(by="as_at_range.from_date").reset_index(drop=True)
result_df


,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
0,"780,360,000,000.00",USD,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:27.633368+00:00,2023-03-30 02:42:27.927445+00:00,superseded
1,"780,360,000,000.00",USD,0001-01-01 00:00:00+00:00,2020-01-01 00:00:00+00:00,2023-03-30 02:42:27.927445+00:00,2023-03-30 02:42:28.366669+00:00,superseded
2,"1,200,000,000,000.00",USD,2020-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:27.927445+00:00,2023-03-30 02:42:28.162332+00:00,superseded
3,"1,200,000,000,000.00",USD,2020-01-01 00:00:00+00:00,2021-01-01 00:00:00+00:00,2023-03-30 02:42:28.162332+00:00,NaT,prevailing
4,"1,680,000,000,000.00",USD,2021-01-01 00:00:00+00:00,2022-01-01 00:00:00+00:00,2023-03-30 02:42:28.162332+00:00,NaT,prevailing
5,"1,200,000,000,000.00",USD,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:28.162332+00:00,2023-03-30 02:42:28.752667+00:00,superseded
6,"780,360,000,000.00",USD,2018-12-31 00:00:00+00:00,2020-01-01 00:00:00+00:00,2023-03-30 02:42:28.366669+00:00,NaT,prevailing


In [8]:
result_df.loc[0].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
0,"780,360,000,000.00",USD,0001-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:27.633368+00:00,2023-03-30 02:42:27.927445+00:00,superseded


<img src="./data/History-1.png" width="850"/>

In [9]:
result_df.loc[1].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
1,"780,360,000,000.00",USD,0001-01-01 00:00:00+00:00,2020-01-01 00:00:00+00:00,2023-03-30 02:42:27.927445+00:00,2023-03-30 02:42:28.366669+00:00,superseded


<img src="./data/History-2.png" width="850"/>

In [10]:
result_df.loc[2].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
2,"1,200,000,000,000.00",USD,2020-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:27.927445+00:00,2023-03-30 02:42:28.162332+00:00,superseded


<img src="./data/History-3.png" width="850"/>

In [11]:
result_df.loc[3].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
3,"1,200,000,000,000.00",USD,2020-01-01 00:00:00+00:00,2021-01-01 00:00:00+00:00,2023-03-30 02:42:28.162332+00:00,NaT,prevailing


<img src="./data/History-4.png" width="850"/>

In [12]:
result_df.loc[4].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
4,"1,680,000,000,000.00",USD,2021-01-01 00:00:00+00:00,2022-01-01 00:00:00+00:00,2023-03-30 02:42:28.162332+00:00,NaT,prevailing


<img src="./data/History-5.png" width="850"/>

In [13]:
result_df.loc[5].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
5,"1,200,000,000,000.00",USD,2022-01-01 00:00:00+00:00,9999-12-31 23:59:59.999999+00:00,2023-03-30 02:42:28.162332+00:00,2023-03-30 02:42:28.752667+00:00,superseded


<img src="./data/History-6.png" width="850"/>

In [14]:
result_df.loc[6].to_frame().transpose()

,value.metric_value.value,value.metric_value.unit,effective_range.from_date,effective_range.until_date,as_at_range.from_date,as_at_range.until_date,status
6,"780,360,000,000.00",USD,2018-12-31 00:00:00+00:00,2020-01-01 00:00:00+00:00,2023-03-30 02:42:28.366669+00:00,NaT,prevailing


<img src="./data/History-7.png" width="850"/>

<img src="./data/History-Complete.png" width="1200"/>

<img src="./data/History-Complete-Comparison.png" width="1400"/>

<p style="font-family: Montserrat; color: #203443; font-weight: bold; font-size: 125%">What have we covered?</p>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We retrieved the time series history for our Instrument Property representing Market Capitalisation that we populated in the previous module.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We saw how the Property's values were described by a combination of "Effective" and "AsAt" ranges.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">Using these ranges we were able to construct a graph which allowed us to understand the value of the Property at any point in bi-temporal time.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We saw how we can modify the "AsAt" date on the request to change how much of the Property's history to return.</p></li>
</ul>

<ul>
    <li style="color: #ff5200"><p style="font-family: Montserrat; color: #203443">We discussed how any value which exists at the provided "AsAt" time is "prevailing" while all other values are "superseded".</p></li>
</ul>